In [1]:
# Asymmetric Least Squares
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load CSV file
data = pd.read_csv('/kaggle/input/blood-malaria/blood2hinfected532nm5pp.csv')

# Extract spectral data
spectra = data.iloc[:, 1:]

# Asymmetric Least Squares (ALS)
def asymmetric_least_squares(spectrum, P=0.001, lam=10**-5):
    def als_objective(params):
        baseline = params[0] + params[1] * np.arange(len(spectrum))
        error = spectrum - baseline
        return np.sum(error**2) + lam * np.sum((P * np.diff(error))**2)

    result = minimize(als_objective, [0, 0], bounds=[(-np.inf, np.inf), (0, np.inf)])
    baseline = result.x[0] + result.x[1] * np.arange(len(spectrum))
    return baseline

# Perform baseline correction
baseline_corrected_spectra = spectra.apply(asymmetric_least_squares, axis=0)

# Create new DataFrame
corrected_data = pd.concat([data.iloc[:, 0], baseline_corrected_spectra], axis=1)

# Save corrected data to new CSV file
corrected_data.to_csv('ALS_corrected_data.csv', index=False)